This notebook is a first attempt to translate the FORTRAN90 code from MTV2016 into python
and to replicate the main results of this paper.

In [54]:
import numpy as np
from scipy.stats import lognorm
from numba import jit

import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

In [82]:
#Paramters for the piece-rate version

delta = 0.000226
lamb_e = 0.7048178
lamb_u = 1 # normalisation
k = 22.9931
sigma_z = 0.0
rhorho1 = 4.296953
rho2 = 0.065518
zlearningprob = 0.26398515
b = 1
theta9 = 0.0
theta10 = 0.0
zshockprob = 0.0093854   


#    IF (npar>=10) THEN
#    rho3=theta(9)  
#    rho4=theta(10)  
#    ELSE IF (npar==8) THEN
#    rho3=0.0_8
#    rho4=0.0_8
#    ELSE 

In [56]:
# sigma_z = 0.2
# mu_temp = np.exp(-0.5 * sigma_z**2) # this guarantess that the mean of the distribution is 1

# x = np.linspace(0, 4, 100)
# plt.plot(x, lognorm.cdf(x, sigma_z, loc=0, scale=mu_temp))
# plt.plot(x, lognorm.pdf(x, sigma_z, loc=0, scale=mu_temp))
# plt.show()

In [57]:
# Match shock z.
# First, populating the match productivity shock vector, if zpts_first is >1

zpts_first = 51 # basic grid. 102 in MTV2016
zpts = zpts_first

# zvector are the actual values of the match quality, dim(zpts)
# ztransmatrix is the transition matrix, dim(zpts, zpts)

# Obtain a discrete LOGNORMAL probability distr for z

zprobcdf = np.zeros((zpts))
zprobpdf = np.zeros((zpts))
zvector = np.zeros((zpts))

sigma_z = 0.2 # CHANGE OF PARAMTER: sigma_z must be positive
mu_temp = np.exp(-0.5 * sigma_z**2) # mean of the lognormal distr, s.t. mean of the distr is 1

shape = sigma_z

resolution = 1000 # Resolution should be much higher than no. of bins (resolution=10000)
xstep = 1 / resolution

xvalue = xstep # move along the interval of z's, starting close to 1
tempreal = 0.5 * xstep * lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) # this value is used to weight the z-values
# properly within the bin
bincounter = 1 # start with the first bin
binsize = 1 / (zpts - 1) # binsize

# zprobcdf contains the probabilities of the idiosyncratic shock, dimension zpts

# First fill all but the last coordinate
while bincounter < zpts - 1:
    if lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) >= ((bincounter + 0) * binsize): # if the bin is full..
        zprobcdf[bincounter] = lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) # add to the cdf
        if bincounter == 1:
            zprobpdf[bincounter] = zprobcdf[bincounter]
            zvector[bincounter] = tempreal / zprobpdf[bincounter]
        else:
            zprobpdf[bincounter] = zprobcdf[bincounter] - zprobcdf[bincounter - 1]
            zvector[bincounter] = tempreal / zprobpdf[bincounter]
        
        tempreal = 0 # reset the average counter 
        bincounter += 1
        
#         while lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) >= (bincounter * binsize):
#             zvector[bincounter] = xstep
#             zprobcdf[bincounter] = lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp)
#             zprobpdf[bincounter] = 0
#             bincounter += 1
            
    else:
        while lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) < ((bincounter + 0) * binsize):
            xvalue = xvalue + xstep
            tempreal = tempreal + (xvalue - 0.5 * xstep) * (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) \
                                                        - lognorm.cdf(xvalue - xstep, shape, loc=0, scale=mu_temp))
# # last gridpoint

zprobcdf[zpts-1] = 1
zprobpdf[zpts-1] = 1 - zprobcdf[zpts - 2]

tempreal = 0
xstep = xvalue / (zpts * 5)

counter1 = 1
while counter1 < 11:
    xvalue = xvalue + xstep
    tempreal = tempreal + (xvalue - 0.5 * xstep) * (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) \
                                                        - lognorm.cdf(xvalue - xstep, shape, loc=0, scale=mu_temp))
    counter1 += 1
    
zvector[zpts-1] = tempreal / (lognorm.cdf(xvalue, shape, loc=0, scale=mu_temp) - zprobcdf[zpts - 2])

zvector[0] = 1 # set the first entry equal to the expected value

In [58]:
# Calculate the mean and renormalize the zvector

tempreal = 0

for zcnt in range(0, zpts):
    tempreal = tempreal + zprobpdf[zcnt] * zvector[zcnt]
    
zvector[:] = zvector[:] / tempreal

In [59]:
# Z-PRODUCTIVITY TRANSITION MATRICES

# INITIAL LEARNING, same for all options

ztransmatrix = np.zeros((zpts, zpts))
zlearningprob = 0.26398515 # from MTV2016

ztransmatrix[0, 0] = 0
for zcnt in range(1, zpts):
    ztransmatrix[0, zcnt] = zlearningprob * zprobpdf[zcnt]
    
tempreal = np.sum(ztransmatrix[0, :])
ztransmatrix[0, 0] = 1 - tempreal

# Permanent z's once learned
for zcnt in range(1, zpts):
    ztransmatrix[zcnt, zcnt] = 1

In [60]:
# Worker productivity - ALL WORKER IDENTICAL WHEN BORN. Otherwise, set ypts = 20 (MTV2016 sub-option)

tmax = 576 # 65-18=48 years, i.e. 48*12=576 months
ypts = 1

y = np.zeros((ypts))
yprobcdf = np.zeros((ypts))
yprobpdf = np.zeros((ypts))
prod = np.zeros((ypts, tmax))

y[:] = 1
yprobcdf[0] = 1
yprobpdf[0] = 1

@jit
def pfunction(exper, rhorho1, rho2):
    output = 1 - rhorho1 + rhorho1 * (exper + 1)**rho2
    return output

# ALLOCATE WORKER PRODUCTIVITY MATRIX, which incorporates both human capital and  worker idiosyncratic productivity
for ycnt in range(0, ypts):
    for exper in range(0, tmax):
        prod[ycnt, exper] = y[ycnt] * pfunction(exper, rhorho1, rho2)  

In [61]:
# Discrete MArkov chain approximation of continuous AR(1) process of aggregate productivity 

from quantecon import tauchen

rho = 0.7 # autocorrelation of AR(1) (Wee)
sigma_u = 0.0165 # standard deviation of AR(1), (Shimer 2005)

aggshock = tauchen(rho, sigma_u, m=3, n=3)

# aggshock.P # shows transition matrix
# aggshock.state_values # associated state space
# aggshock.stationary_distributions # shows the stationary distribution

In [62]:
# Define job-finding rate (CD)

eta = 0.5

@jit
def pf(theta):
    pf = theta**eta
    return pf

In [63]:
# INITIALIZE

ppts = 1 # no aggregate shock in MTV2016
ypts = 1 # no ex-ante heterogeneity in MTV016
tmx = 576
expts = tmax
zpts = 51

ppts = 1
pvector = np.ones((ppts))

VU = np.zeros((ppts, ypts, expts, 2))             # Value of unemployment
VE = np.zeros((ppts, ypts, expts, zpts, 2))       # Joint value of employment, conditional on z, t
EV = np.zeros((ppts, ypts, expts, 2))             # Expected value of a new match
poliDW = np.zeros((ppts, ypts, expts, zpts, tmax)) 
Dmax = np.zeros((ppts, ypts, expts, zpts, 2))     # Expected surplus value of search. !!!second argument zpts refers to current promised value
DmaxU = np.zeros((ppts, ypts, expts, 2))          # Expected surplus value of search for unemployed
S = np.zeros((ppts, ypts, expts, zpts, 2))        # Expected productivity of the match
phiu = np.zeros((ppts, ypts, expts, tmax))        # piece rate offered to previously unemployed
phie = np.zeros((ppts, ypts, expts, zpts, tmax))  # piece rate offered to previously employed at z
duration = np.zeros((ppts, ypts, expts, zpts, 2)) # duration, match with current z, not necess. initial z
JF_E = np.zeros((ppts, ypts, expts, zpts, tmax))  # Prob of succesful application *from* a match z,t
JF_U = np.zeros((ppts, ypts, expts, tmax))        # Prob of succesful application *from* unemployment at t

In [64]:
# LAST PERIOD, t=T

# 1) Unemployment

VU[:, :, :, 1] = b

# 2) Match values

for zcnt in range(0, zpts):
    for exper in range(0, tmax):
        for ycnt in range(0, ypts):
            for pcnt in range(0, ppts):
                VE[pcnt, ycnt, exper, zcnt, 1] = pvector[pcnt] * zvector[zcnt] * prod[ycnt, exper] # zcnt = 1 computes E[z]

In [67]:
# 3) EXPECTED MATCH VALUE, conditional on y-heterogeneity and experience

# 4) DETERMINING  TH(z,t), Dmax(t, V), DmaxU(t)

# Here we use the planner's problem: 

# FOC of maximizing the surplus value of the application: k=η*θ**(η-1)[EV(tmax)-V(zcnt, tmax)]
#                  from  (1) k=q(θ)[EV - X], where X is the value applied to by the worker
#                  and   (2) Dmax(t,V)=max p(θ)[X-V]
#                  SOLVE FOR X, and take a derivative wrt to θ to find the solution


# 4a1. UNEMPLOYED SEARCHERS without screening

for exper in range(0, tmax):
    for ycnt in range(0, ypts):
        for pcnt in range(0, ppts):
            if VU[pcnt, ycnt, exper, 1] < VE[pcnt, ycnt, exper, 0, 1]:
                thetatemp = min(((eta * (VE[pcnt, ycnt, exper, 0, 1] - VU[pcnt, ycnt, exper, 1])) / k)**(1 / (1 - eta)), 1)
                # theta= [η(VE-VU)/k]**(1/(1-η)), from the FOC in efficient case: f'(θ)(EV-VU)=k, with f(θ)=θ**η, eq. (9)
                      
                DmaxU[pcnt, ycnt, exper, 1] = pf(thetatemp) * (VE[pcnt, ycnt, exper, 0, 1] - \
                                                                      VU[pcnt, ycnt, exper, 1]) - k * thetatemp
                # dmaxU = f(θ)(VE-VU)- k θ, eq. (7)
                      
                if DmaxU[pcnt, ycnt, exper, 1] < 0:
                    print("surplus cannot be lower than 0")
                
                JF_U[pcnt, ycnt, exper, tmax-1] = pf(thetatemp)
                      
            else:
                JF_U[pcnt, ycnt, exper, tmax-1] = 0 # around experience 60 starts to become positive
                DmaxU[pcnt, ycnt, exper, 1] = 0

In [68]:
# 4b1. EMPLOYED/NEW MATCHES SEARCHERS without screening

for zcnt in range(0, zpts):
    for exper in range(0, tmax):
        for ycnt in range(0, ypts):
            for pcnt in range(0, ppts):
                if VE[pcnt, ycnt, exper, zcnt, 1] < VE[pcnt, ycnt, exper, 0, 1]: # VE[:, :, :, E(z), :]
                    thetatemp = min(((eta * (VE[pcnt, ycnt, exper, 0, 1] - VE[pcnt, ycnt, exper, zcnt, 1])) / k)**(1 / (1 - eta)), 1)

                    Dmax[pcnt, ycnt, exper, zcnt, 1] = pf(thetatemp) * (VE[pcnt, ycnt, exper, 0, 1] - \
                                                                          VE[pcnt, ycnt, exper, zcnt, 1]) - k * thetatemp

                    if Dmax[pcnt, ycnt, exper, zcnt, 1] < 0:
                        print("surplus cannot be lower than 0")

                    JF_E[pcnt, ycnt, exper, zcnt, tmax-1] = pf(thetatemp)

                else:
                    JF_E[pcnt, ycnt, exper, zcnt, tmax-1] = 0 # Somewehre around z = zpts/2 the expected VE starts being smaller than VE
                    Dmax[pcnt, ycnt, exper, zcnt, 1] = 0

In [69]:
# 5) PARTICIPATION DECISION

# See if nobody prefers to be unemployed, even though that entails no search

for zcnt in range(0, zpts):
    for exper in range(0, tmax):
        for ycnt in range(0, ypts):
            for pcnt in range(0, ppts):
                tempreal = VE[pcnt, ycnt, exper, zcnt, 1] + lamb_e * max(Dmax[pcnt, ycnt, exper, zcnt, 1], 0)
                
                if tempreal > VU[pcnt, ycnt, exper, 1]:
                    poliDW[pcnt, ycnt, exper, zcnt, 1] = 1 # 1 if participation continues
                else:
                    poliDW[pcnt, ycnt, exper, zcnt, 1] = 0 # 0 if participation ends

In [70]:
# 6) VALUE OF THE MATCH, expected duration

for zcnt in range(0, zpts):
    for exper in range(0, tmax):
        for ycnt in range(0, ypts):
            for pcnt in range(0, ppts):
                S[pcnt, ycnt, exper, zcnt, 1] = pvector[pcnt] * zvector[zcnt] * prod[ycnt, exper]
                duration[pcnt, ycnt, exper, zcnt, 1] = 1 # expected duration at the production stage

In [81]:
# 7) Fixed wages and piece rates 

# In the last period these two wage setting mechanisms are the same

# no screening

stevens = 0
fixedwage = 1
piecerate = 0

for exper in range(0, tmax):
    for ycnt in range(0, ypts):
        for pcnt in range(0, ppts):
            
            if stevens == 1:
                phiu[pcnt, ycnt, exper, tmax - 1] = pvector[pcnt] * zvector[0] * prod[ycnt, exper] - \
                k / JF_U[pcnt, ycnt, exper, tmax - 1]**((eta - 1) / eta)
                # w = production - k / q(θ)
                
            elif fixedwage == 1:
                phiu[pcnt, ycnt, exper, tmax - 1] = (S[pcnt, ycnt, exper, 0, 1] - \
                k / JF_U[pcnt, ycnt, exper, tmax - 1]**((eta - 1) / eta)) / \
                duration[pcnt, ycnt, exper, 0, 1]
                # w = (S - k / q(θ)) / duration
                
            elif piecerate == 1:
                phiu[pcnt, ycnt, exper, tmax - 1] = 1 - \
                k / ((JF_U[pcnt, ycnt, exper, tmax - 1]**((eta - 1) / eta)) * \
                S[pcnt, ycnt, exper, 0, 1])
                # w = (1 - k / (q(θ) * S))

In [90]:
# NLFPROBS will generate probabilities that can be used to see people transit into NLF
# NLFVECTOR - calculate the survival probability for each time t

rr = 1.04**(1 / 12) # gross real interest rate (based on 4% annual)
bt = 1 / 1.04**(1 / 12) # corresponding discount factor

nlfvector = np.ones((tmax)) # vector of exit rates

# Homogenous rate
# nlfvector[:] = 1

# Heterogenous rates
@jit
def betafunction(tt):
    if tt > 400 and tt <= 564:
        x = 0.0919441 - 0.0014106 * tt + 7.56e-06 * tt**2 - 1.70e-08 * tt**3 + 1.39e-11 * tt**4.0 - 0.004
    else:
        x = 0
    value = max(x, 0)
    return value

for counter2 in range(12, tmax):
    nlfvector[counter2] = 1 - betafunction(counter2 - 12)

In [96]:
# BACKWARD INDUCTION

# 1) Unemployment

# VU, EV, Dmax, DmaxU refer to the next period - VE to this period

ptransmatrix = np.ones((1,1))

for tcnt in range(0, tmax): # time
    for exper in range(0, tmax):
        for ycnt in range(0, ypts):
            for pcnt in range(0, ppts):
                tempreal = 0

                for pcnt2 in range(0, ppts):
                    tempreal = tempreal + ptransmatrix[pcnt, pcnt2] * (VU[pcnt, ycnt, exper, 1] + lamb_u * max(DmaxU[pcnt, ycnt, exper, 1], 0))

                VU[pcnt, ycnt, exper, 0] = b + bt * nlfvector[tcnt+1] * tempreal

                # VU[:, :, :, 0] now refers to the current period
                # VU, VE, EV, Dmax, DmaxU[:, :, :, 1] refer to the next period
                # experience increases after production this period

IndexError: index 576 is out of bounds for axis 0 with size 576